In [5]:
import pyopencl
import numpy

# How to Program Devices

## Selecting platforms &/ devices
Looking at what is available, then selecting the NVIDIA one.

In [6]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Intel(R) OpenCL
Portable Computing Language
NVIDIA CUDA


In [7]:
nvidia_platform = [platform 
                   for platform in pyopencl.get_platforms() 
                   if platform.name == "NVIDIA CUDA"][0]
nvidia_devices = nvidia_platform.get_devices()

## Building Programs
Using the PyOpenCL bindings to create an OpenCL context, then declaring OpenCL kernel code, and compiling it.

The code is for a simple vector sum, i.e. $\vec{c} = \vec{a} + \vec{b}$

In [8]:
nvidia_context = pyopencl.Context(devices=nvidia_devices)

program_source = """
      kernel void sum(global float *a, 
                      global float *b,
                      global float *c){
        int gid = get_global_id(0);
        c[gid] = a[gid] + b[gid];
      }
    """
nvidia_program_source = pyopencl.Program(nvidia_context, program_source)
nvidia_program = nvidia_program_source.build()

In [9]:
program_kernel_names = nvidia_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print("Kernel Names: {}".format(program_kernel_names))

Kernel Names: sum


## Running Programs
Actually running the code using the Python bindings. 

Some memory and data needs to be setup first, but I'll elaborate on this later.

In [10]:
def run_ocl_kernel(queue, 
                   kernel, 
                   global_size,
                   input_tuples,
                   output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (array, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=array, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (arr,buffer) in input_tuples] 
    kernel_arguments += [buffer for (arr,buffer) in output_tuples]
        
    kernel(queue,
           global_size,
           local_size,
           *kernel_arguments)

    # copying data off the device
    for (arr, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=arr)
        
    # waiting for everything to finish
    queue.finish()

In [11]:
def check_sum_results(a,b,c):
    c_ref = a + b
    err = numpy.abs(c - c_ref)
    if((err.sum() > 0.0).any()): 
        print("result does not match")
    else: 
        print("result matches!")   

In [12]:
# Synthetic data setup
N = int(2**20)
a = numpy.random.rand(N).astype(numpy.float32)
b = numpy.random.rand(N).astype(numpy.float32)
c = numpy.empty_like(a)

# Device Memory setup
a_nvidia_buffer = pyopencl.Buffer(nvidia_context,
                                  flags=pyopencl.mem_flags.READ_ONLY, 
                                  size=a.nbytes)
b_nvidia_buffer = pyopencl.Buffer(nvidia_context, 
                                  flags=pyopencl.mem_flags.READ_ONLY, 
                                  size=b.nbytes)
c_nvidia_buffer = pyopencl.Buffer(nvidia_context, 
                                  flags=pyopencl.mem_flags.WRITE_ONLY, 
                                  size=c.nbytes)

In [13]:
nvidia_queue = pyopencl.CommandQueue(nvidia_context)

input_tuples = ((a, a_nvidia_buffer), (b, b_nvidia_buffer), )
output_tuples = ((c, c_nvidia_buffer),)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)

In [14]:
check_sum_results(a, b, c)

result matches!


In [15]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)

2.14 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


***Suggestion***: Characterise the OpenCL devices you can get access to, stepping up the size of $N$ by some factor (power of 2 is easy to reason about). 

# How to Manipulate Memory

## Using global memory
Differentiating between different memory flags, and defining some utility functions to help set up memories.

In [16]:
def create_input_memory(context, input_arrays):
    return [
        (array, pyopencl.Buffer(context,
                                flags=pyopencl.mem_flags.READ_ONLY, 
                                size=array.nbytes))
        for array in input_arrays
    ]

In [17]:
def create_output_memory(context, output_arrays):
    return [
        (array, pyopencl.Buffer(context,
                                flags=pyopencl.mem_flags.WRITE_ONLY, 
                                size=array.nbytes))
        for array in output_arrays
    ]

In [18]:
def cleanup_memories(tuples):
    for (_, buffer) in tuples:
        buffer.release()

In [19]:
a = numpy.random.rand(N).astype(numpy.float32)
b = numpy.random.rand(N).astype(numpy.float32)
c = numpy.empty_like(a)

# Device Memory setup
input_tuples = create_input_memory(nvidia_context, (a,b,))
output_tuples = create_output_memory(nvidia_context, (c,),)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples)
check_sum_results(a,b,c)

result matches!


## Using local and private memory
Exploring the use of more localised memories to optimise performance. 

Most of the difference lives in the kernels, which are defined in a separate file.

In [20]:
def build_program(ocl_file, context, build_options=[]):
    with open(ocl_file,"r") as opencl_file:
        program_source_code = opencl_file.read()
        program_source = pyopencl.Program(context, program_source_code)
        program = program_source.build(options=build_options)
        
        return program

In [21]:
batch_size = 16
wg_size = 32
power = 1
build_options = ["-D","BATCH_SIZE={}".format(batch_size),
                 "-D","WG_SIZE={}".format(wg_size),
                 "-D","POW={}".format(power)]

nvidia_program = build_program("vector_sum.cl", nvidia_context, build_options)
run_ocl_kernel(nvidia_queue, nvidia_program.sum, (N,), input_tuples, output_tuples, (wg_size,))
check_sum_results(a,b,c)

result matches!


In [22]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum_batched, (N//batch_size,), input_tuples, output_tuples)

2.97 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16, (N//16,), input_tuples, output_tuples)

2.02 ms ± 22.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum_batched_private, (N//batch_size,), input_tuples, output_tuples)

2.2 ms ± 37.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16_local, (N//16,), input_tuples, output_tuples)

2 ms ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit (a+b)

503 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
power = 1000
build_options = ["-D","BATCH_SIZE={}".format(batch_size),
                 "-D","WG_SIZE={}".format(wg_size),
                 "-D","POW={}".format(power)]

nvidia_program = build_program("vector_sum.cl", nvidia_context, build_options)

In [35]:
%timeit run_ocl_kernel(nvidia_queue, nvidia_program.sum16_pow, (N//16,), input_tuples, output_tuples)

3.74 ms ± 421 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
%timeit (a+b)**power

/home/gordon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in power
  """Entry point for launching an IPython kernel.


27.7 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# How to Do Things in Parallel

## Device characteristics
Using the host code API to inspect the characteristics of the device.

In [25]:
intel_platform = [platform 
                  for platform in pyopencl.get_platforms() 
                  if platform.name == "Portable Computing Language"][0]
intel_devices = [intel_platform.get_devices()[0]]

In [26]:
name_properties = {
    "Device Name":pyopencl.device_info.NAME,
    "Device Platform":pyopencl.device_info.PLATFORM,
    "Device Type":pyopencl.device_info.TYPE
}

processing_properties = {
    "Available Compute Units": pyopencl.device_info.MAX_COMPUTE_UNITS,
    "Clockrate": pyopencl.device_info.MAX_CLOCK_FREQUENCY
}

memory_properties = {
    "Available Global Memory": pyopencl.device_info.GLOBAL_MEM_SIZE,
    "Available Constant Memory": pyopencl.device_info.MAX_CONSTANT_BUFFER_SIZE,
    "Available Local Memory" : pyopencl.device_info.LOCAL_MEM_SIZE
}

device_types = {
    pyopencl.device_type.CPU:"CPU",
    pyopencl.device_type.GPU:"GPU"
}

In [27]:
for device in (intel_devices):
    #print out all of the device name properties, except the device type
    for property_name in sorted(name_properties.keys() - {"Device Type"}):
        property_string_args = (property_name,device.get_info(name_properties[property_name]))
        print("{}: {}".format(*property_string_args))
        
    #look up the device type
    print("Device Types: {}".format(device_types[device.get_info(name_properties["Device Type"])]))
    
    #print out all of the processing properties
    for property_name in sorted(processing_properties.keys()):
        property_string_args = (property_name,device.get_info(processing_properties[property_name]))
        print("{}: {}".format(*property_string_args))
    
    #print out all of the memory properties
    for property_name in sorted(memory_properties.keys()):
        property_string_args = (property_name,device.get_info(memory_properties[property_name]))
        print("{}: {}".format(*property_string_args))
        
    print("\n")

Device Name: pthread-Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Device Platform: <pyopencl.Platform 'Portable Computing Language' at 0x7fa60919e020>
Device Types: CPU
Available Compute Units: 8
Clockrate: 3800
Available Constant Memory: 4194304
Available Global Memory: 14515052544
Available Local Memory: 4194304




In [28]:
!clinfo

Number of platforms                               3
  Platform Name                                   Intel(R) OpenCL
  Platform Vendor                                 Intel(R) Corporation
  Platform Version                                OpenCL 2.0 
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_depth_images cl_khr_fp64 cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_icd cl_khr_image2d_from_buffer cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_spir
  Platform Extensions function suffix             INTEL

  Platform Name                                   Portable Computing Language
  Platform Vendor                                 The pocl project
  Platform Version                                OpenCL 1.2 pocl 1.1 None+Asserts, LLVM 6.0.0, SPIR, SLEEF, DISTRO, POCL_DEBUG
  Platform Profile               

  Preferred work group size multiple              128
  Preferred / native vector sizes                 
    char                                                 1 / 32      
    short                                                1 / 16      
    int                                                  1 / 8       
    long                                                 1 / 4       
    half                                                 0 / 0        (n/a)
    float                                                1 / 8       
    double                                               1 / 4        (cl_khr_fp64)
  Half-precision Floating-point support           (n/a)
  Single-precision Floating-point support         (core)
    Denormals                                     Yes
    Infinity and NANs                             Yes
    Round to nearest                              Yes
    Round to zero                                 No
    Round to infinity                             No
    

  Preferred work group size multiple              32
  Warp size (NV)                                  32
  Preferred / native vector sizes                 
    char                                                 1 / 1       
    short                                                1 / 1       
    int                                                  1 / 1       
    long                                                 1 / 1       
    half                                                 0 / 0        (n/a)
    float                                                1 / 1       
    double                                               1 / 1        (cl_khr_fp64)
  Half-precision Floating-point support           (n/a)
  Single-precision Floating-point support         (core)
    Denormals                                     Yes
    Infinity and NANs                             Yes
    Round to nearest                              Yes
    Round to zero                                 Yes
    

## Workitems vs Workgroups
Exploring different types of parallelism, using a more involved example, which is the Java string hash function.

In [29]:
intel_context = pyopencl.Context(devices=intel_devices)
intel_queue = pyopencl.CommandQueue(intel_context)

In [30]:
!wget https://s3-eu-west-1.amazonaws.com/word-count-test-bucket/shakespeare_words.txt

--2018-07-07 10:00:28--  https://s3-eu-west-1.amazonaws.com/word-count-test-bucket/shakespeare_words.txt
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.20.12
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.20.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5458199 (5.2M) [text/plain]
Saving to: ‘shakespeare_words.txt.1’

shakespeare_words.t 100%[===================>]   5.21M   574KB/s    in 16s     

2018-07-07 10:00:47 (327 KB/s) - ‘shakespeare_words.txt.1’ saved [5458199/5458199]



In [37]:
def get_word_arrays_from_file(filename, N_words):
    """Reads a text file into numpy arrays.
    
    Keyword arguments:
    filename -- the file to read from
    N_words -- maximum number of words to return
    
    Returns:
    data -- numpy.Array ASCII with a single entry containing all of 
            the non-whitespace characters in the file.
    offsets -- numpy.Array containing an integer element 
                for each word's offset
    lengths -- numpy.Array containing an integer element 
                for each word's length  
    """
    with open(filename,"r") as data_file:
        # Reading data from file, and making it into one big string
        data = [line[:-1] for line in data_file]
        data_string = "".join(data)
    
        # Getting the lengths of each word
        lengths = [len(word) for word in data]
    
        # Getting the start of each word
        offsets = [0]
        for word in data[:-1]:
            offsets += [offsets[-1] + len(word)]
        
        # Testing that the offsets and counts are correct
        for i,word in enumerate(data):
            temp_word = data_string[offsets[i]:offsets[i]+lengths[i]] 
            if(word != temp_word): 
                print("Problem :",word,"!=",temp_word)
                raise("Data mismatch!")
            
        # Converting data into numpy array
        offsets_array = numpy.array(offsets[:N_words], dtype=numpy.int32)
        lengths_array = numpy.array(lengths[:N_words], dtype=numpy.int32)
        
        end_last_word = offsets_array[-1] + lengths_array[-1]
        byte_data_string = data_string[:end_last_word].encode("ascii","ignore")
        data_array = numpy.array(byte_data_string)
            
    return data_array, offsets_array, lengths_array

In [38]:
# Source data
N_words = 2**20
words, offsets, lengths = get_word_arrays_from_file("shakespeare_words.txt", N_words)

In [46]:
def create_data_buffers(context, words, offsets, lengths):
    """Creates a read only PyOpenCL buffers for a numpy.Array
    
    Keyword arguments:
    context -- pyopencl.Context that buffer will be created in.
    words -- numpy.Array of words data
    offsets -- numpy.Array of offsets data
    lengths -- numpy.Array of lengths data
    
    Returns:
    words_buffers -- pyopencl.Buffer in context for words array
    offsets_buffers -- pyopencl.Buffer in context for offsets array
    lengths_buffers -- pyopencl.Buffer in context for lengths array
    """
    ro_mem_flags = pyopencl.mem_flags.READ_ONLY | pyopencl.mem_flags.ALLOC_HOST_PTR
    
    # Source buffers
    buffers = [pyopencl.Buffer(context,
                               flags=ro_mem_flags,
                               size=array.nbytes) 
               for array in (words,offsets,lengths)]
    
    wo_mem_flags = pyopencl.mem_flags.WRITE_ONLY
    output_size = offsets.nbytes #int per word
    
    # Destination buffers
    buffers += [pyopencl.Buffer(context,
                                flags=wo_mem_flags,
                                size=output_size)]
    
    return tuple(buffers)

In [47]:
# Intel Buffers
intel_buffers = create_data_buffers(intel_context, words, offsets, lengths)

In [87]:
intel_program = build_program("java_hash.cl", intel_context)

In [88]:
def hash_words(queue, program, data_arrays, data_buffers, local_size=(32,)):
    """Copy data onto OpenCL device, does the hashing operation, copies result off.
    
    Keyword arguments:
    queue -- pyopencl.Queue 
    program -- compiled pyopencl.Program 
    data_arrays -- Source data arrays
    data_buffers -- pyopencl.Buffers to use in pyopencl problem
    hashes -- Empty numpy.Array for results
    
    Returns:
    hashes -- Copy of numpy.Array of ints, containing the results
    """
    words, offsets, lengths = data_arrays
    words_buffer, offsets_buffer, lengths_buffer, hashes_buffer = data_buffers
    
    # Copying data onto the device
                     # Words
    copyon_events = [pyopencl.enqueue_copy(queue,
                                           src=words,
                                           dest=words_buffer),
                     # Offsets
                     pyopencl.enqueue_copy(queue,
                                           src=offsets,
                                           dest=offsets_buffer),
                     # Lengths
                     pyopencl.enqueue_copy(queue,
                                           src=lengths,
                                           dest=lengths_buffer)]
    
    # Hashing the words
    kernel_event = program.java_hash_cached(queue,
                                            (offsets.size,), #global size
                                            local_size, #local size
                                            words_buffer,offsets_buffer,lengths_buffer,hashes_buffer, #buffers
                                            wait_for=copyon_events)
    
    
    # Copying data off the device
    hashes = numpy.empty(offsets.shape,dtype=numpy.int32)
    copyoff_event = pyopencl.enqueue_copy(queue,
                                          src=hashes_buffer,
                                          dest=hashes,
                                          wait_for=[kernel_event])
    
    #wait for copy-off to finish
    copyoff_event.wait()
    
    return hashes

In [89]:
intel_hashes = hash_words(intel_queue, intel_program, 
                          (words, offsets, lengths,), 
                          intel_buffers)

In [82]:
def ref_java_hash(word):
    """Hashes a string into 32-bit integer value, as per the Java hash algorithm"""
    h = 0
    for c in word:
        h *= 31
        h += c
        h &= 0xFFFFFFFF
        
    return ((h + 0x80000000) & 0xFFFFFFFF) - 0x80000000

In [90]:
temp_words_raw = str(words)[2:-1]
temp_words = [temp_words_raw[offset:(offset+length)].encode('ascii') for offset,length in zip(offsets[:N],lengths[:N])]
ref_hashes = numpy.fromiter(map(ref_java_hash,temp_words),numpy.int32)

for i,(intel_hash,ref_hash) in enumerate(zip(intel_hashes, ref_hashes)):
    if(intel_hash != ref_hash): 
        print(i,"Problem!",intel_hash,ref_hash)

In [93]:
%timeit hash_words(intel_queue, intel_program, (words, offsets, lengths,), intel_buffers, local_size=(32,))

4.48 ms ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
%timeit numpy.fromiter(map(ref_java_hash,temp_words),numpy.int32)

424 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
import numba

In [85]:
@numba.jit(nopython=True)
def ref_java_hash_numba(word):
    """Hashes a string into 32-bit integer value, as per the Java hash algorithm"""
    h = 0
    for c in word:
        h *= 31
        h += c
        h &= 0xFFFFFFFF
        
    return ((h + 0x80000000) & 0xFFFFFFFF) - 0x80000000

In [86]:
%timeit numpy.fromiter(map(ref_java_hash_numba,temp_words),numpy.int32)

292 ms ± 6.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
